In [108]:
import tweepy
import csv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os.path

consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

data_sufix = 'data/'

In [109]:
if not os.path.isdir(data_sufix):
    os.system('mkdir '+data_sufix)

In [110]:
def create_files_if_not_present(hashtag):
    if not os.path.isfile(data_sufix+hashtag+'-tweet.csv'):
        with open(data_sufix+hashtag+'-tweet.csv', 'a') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['tweet_id', 'tweet_text', 'tweet_created_at', 
                               'tweet_retweet_count', 'tweet_favorite_count', 'user_id'])
        csvfile.close()
    if not os.path.isfile(data_sufix+hashtag+'-retweet.csv'):
        with open(data_sufix+hashtag+'-retweet.csv', 'a') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['retweet_id','source_tweet_id','retweet_text','retweet_created_at', 'user_id'])
        csvfile.close()

In [111]:
def write_user_to_csv(user):
    with open(data_sufix+'user.csv', 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([user.id, user.name, user.location, user.friends_count, user.followers_count])
    csvfile.close()

In [112]:
def write_tweet_to_csv(tweet, hashtag):
    with open(data_sufix+hashtag+'-tweet.csv', 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([tweet.id, tweet.text.encode("utf-8"), tweet.created_at, 
                            tweet.retweet_count, tweet.favorite_count, tweet.user.id])
    csvfile.close()

In [113]:
def write_retweet_to_csv(retweet, hashtag):
    with open(data_sufix+hashtag+'-retweet.csv', 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([retweet.id, retweet.retweeted_status.id, retweet.text.encode("utf-8"),
                            retweet.created_at, retweet.user.id])
    csvfile.close()

In [114]:
def write_all_to_file(tweet, hashtag):
    with open(data_sufix+'all-'+hashtag+'.csv', 'a') as file:
        writer = csv.writer(file)
        writer.writerows(tweet.__dict__.items())
    file.close()

In [115]:
if not os.path.isfile(data_sufix+'user.csv'):
        with open(data_sufix+'user.csv', 'w') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['user_id','user_name','user_location','user_friends_count','followers_count'])
        csvfile.close()
        
userData = pd.read_csv(data_sufix+'user.csv', usecols=['user_id'])
already_collected_users = [user_id for user_id in userData.user_id]

In [ ]:
hashtags = ['h1b', 'usvisa', 'immigrationreform', 'illegalImmigration', 'immigration',
'immigrantcaravan', 'caravaninvasion', 'buildthewall', 'illegalaliens', 'birthrightcitizenship']

In [ ]:
for hashtag in hashtags:
    
    create_files_if_not_present(hashtag)
    
    tweetData = pd.read_csv(data_sufix+hashtag+'-tweet.csv', usecols=['tweet_id'])
    already_collected_tweets = [tweet_id for tweet_id in tweetData.tweet_id]
    
    searched_tweets = [status for status in tweepy.Cursor(api.search, q='#'+hashtag, lang="en", since='2018-10-27', until='2018-10-31').items()]
    
    tweet_count = 0
    retweet_count = 0
    ignored_count = 0
    
    for tweet in reversed(searched_tweets): # reversed to process older tweets first
        action = False
        write_all_to_file(tweet, hashtag)
        if hasattr(tweet,'retweeted_status'): # it is a retweet
            if(tweet.retweeted_status.id in already_collected_tweets): # base tweet has already been saved
                write_retweet_to_csv(tweet, hashtag)
                action = True
                retweet_count += 1
            else: # if base tweet not saved - ignore retweet
                ignored_count += 1
        else: # it is not a retweet
            already_collected_tweets.append(tweet.id)
            write_tweet_to_csv(tweet, hashtag)
            action = True
            tweet_count += 1
        if(tweet.user.id not in already_collected_users and action is True):
            already_collected_users.append(tweet.user.id)
            write_user_to_csv(tweet.user)
            
    print('For', hashtag, '(', len(searched_tweets), ') :\ntweets - ', tweet_count,
          '\nretweets - ', retweet_count, '\nignored - ', ignored_count)

In [ ]:
# Retweeted metrics:
# tweet.retweeted - Indicates whether this Tweet has been Retweeted by the authenticating user
# tweet.retweet_count - Number of times this (original, source!!!) Tweet has been retweeted
# tweet.retweeted_status - Retweets can be distinguished from typical Tweets by the existence of a retweeted_status 
#                          attribute. This attribute contains a representation of the original Tweet that was retweeted.